Maxwell solver for PEC bodies (EFIE)
=============================
**keys**: Maxwell single layer potential, EFIE, indirect ansatz, MoM, PEC scattering

In [ ]:
import sys
sys.path.append("../build/")
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve.krylovspace import CG, GMRes

We consider a perfect conducter $\Omega \subset \mathbb R^3$ and a plane wave $\boldsymbol E_{\mathrm{inc}} $ with tangential trace $\gamma_R \boldsymbol E_{\mathrm{inc}} = -\boldsymbol m \in \boldsymbol H^{-\frac12}\left( \mathrm{curl}_\Gamma, \Gamma\right)\,.$ The incoming wave thus induces a scattered electric field $\boldsymbol E$ which propagates into $\Omega^c$. The scattered electric field solves the following boundary value problem: 

$$ \left\{ \begin{array}{rcl l} \mathbf{curl} \, \mathbf{curl}\, \boldsymbol E - \kappa^2 \, \boldsymbol E &=& \boldsymbol 0, \quad &\textnormal{in } \Omega^c \subset \mathbb R^3\,,\\ \gamma_R \,\boldsymbol E &=& \boldsymbol m, \quad & \textnormal{on }\Gamma \\ \left\| \mathbf{curl} \, \boldsymbol E( x) - i\,\omega\,\epsilon \, \boldsymbol E( x)\right\| &=& \mathcal O\left( \displaystyle \frac{1}{\| x\|^2}\right), &\textnormal{for} \; \|x\| \to \infty\,.\end{array} \right. $$ 

and a possible representation reads

$$ \boldsymbol E(x) = \underbrace{ \kappa \, \int_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}\sigma_y + \frac{1}{\kappa} \nabla \int_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{SL} (\boldsymbol j) } } \,.$$ 

The density $\boldsymbol j$ is determined by the boundary element method, i.e. the numerical solution of the variational formulation 

$$ \begin{array}{c} \forall \, \boldsymbol v\in H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma): \quad \left\langle \,\mathrm{SL} (\boldsymbol j),\, \boldsymbol v \right\rangle_{-\frac12} = \left\langle \boldsymbol m , \boldsymbol v\right\rangle_{-\frac12} \,. \end{array}$$ 

In the enineering community, the approximation scheme is also known as **method of moments** (MoM) and the resulting equation for $\boldsymbol j$ is called the **EFIE**, the **electric field integral equation**. 

Define the geometry of the perfect conductor $\Omega$ and create a mesh:

In [ ]:
#shape = Box((-10., -1., -1.), (10., 1., 1.)) + Box((-6., -1, 1.), (0., 1, 2.)) + Box((6., -1., 1.), (7., 1., 3.5))
#mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=2., perfstepsend=meshing.MeshingStep.MESHSURFACE))
mesh = Mesh("ship0.stl")
mesh.Curve(4)
print(mesh.nface)
Draw(mesh)

Next, we create finite element spaces $\boldsymbol H^{-\frac12}(\mathrm{curl}_\Gamma, \Gamma)$ and $\boldsymbol H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma)$:

In [ ]:
order = 4
fesHDiv = HDivSurface(mesh, order=order, complex=True)
uHDiv, vHDiv = fesHDiv.TnT()
fesHCurl = HCurl(mesh, order=order, complex=True)
uHCurl, vHCurl = fesHDiv.TnT()
print("ndof HDiv =", fesHDiv.ndof, "ndof HCurl =", fesHCurl.ndof)

Define the incoming plane wave and compute the given Dirichlet data $\boldsymbol m$: 

In [ ]:
kx = -0.785
ky = - 0.453
kz = - 0.524
kappa = sqrt(kx ** 2 + ky ** 2 + kz ** 2)
E_inc = CF((-0.66, 0.436, 0.112)) * exp(1j * (kx * x + ky * y + kz * z))

n = specialcf.normal(3)
m = -Cross(Cross(n, E_inc), n)
gfm = GridFunction(fesHCurl)
gfm.Set(m, definedon=mesh.Boundaries(".*"), dual=True)
print(sqrt(Integrate(Norm(m - gfm) ** 2, mesh.Boundaries(".*"), BND)) / sqrt(Integrate(Norm(m) ** 2, mesh.Boundaries(".*"), BND)))
Draw(m[0].real, mesh, draw_vol=False, order=order);

# Compute the right hand side vector:

In [ ]:
#rhs = LinearForm(m * vHDiv.Trace() * ds(bonus_intorder=3)).Assemble()

The discretisation of the above variational formulation leads to a system of linear equations, ie

$$ V\, \mathrm j = \mathrm{rhs}\,,$$

where the single layer operator $V$ on the left is a regular and symmetric matrix.

In [ ]:
j = GridFunction(fesHDiv)
pre = BilinearForm(uHDiv.Trace() * vHDiv.Trace() * ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, kappa, intorder=16, leafsize=200, eta=0., eps=1e-8)
    M = BilinearForm(uHCurl.Trace() * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble()
    rhs = (M.mat * gfm.vec).Evaluate()
    GMRes(A = V.mat, pre=pre, b=rhs, x=j.vec, tol=1e-8, maxsteps=4000, printrates=True)
j.vec[:] *= kappa

In [ ]:
Draw(j[0].real, mesh, draw_vol=False, order=order, min=-0.5, max=0.5);